# Custom Konkani Emotion Detection Model - From Scratch

**Approach:** Build custom LSTM/GRU model trained only on Konkani data  
**No pre-trained models** - pure custom architecture!

## Step 1: Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nGPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Step 2: Install Dependencies

In [ ]:
!pip install -q torch torchtext pandas scikit-learn tqdm
print("✅ Dependencies installed!")

## Step 3: Load Emotion Data

In [ ]:
import pandas as pd
import os

DATASET_PATH = "/kaggle/input/konkani-emotion-data"  # UPDATE THIS

# Load data
df = pd.read_csv(f"{DATASET_PATH}/konkani_emotion_training_data.csv")

print(f"Dataset size: {len(df)}")
print(f"\nEmotion distribution:")
print(df['emotion'].value_counts())
print(f"\nSample:")
print(df.head(3))

## Step 4: Build Custom Vocabulary

In [ ]:
from collections import Counter
import re

# Tokenize Konkani text
def tokenize_konkani(text):
    # Simple word tokenization for Devanagari
    return text.strip().split()

# Build vocabulary from training data
all_tokens = []
for text in df['text']:
    all_tokens.extend(tokenize_konkani(text))

# Count frequencies
token_counts = Counter(all_tokens)

# Build vocab (keep top 10000 words)
vocab_size = 10000
most_common = token_counts.most_common(vocab_size - 2)  # -2 for PAD and UNK

# Create word to index mapping
word2idx = {'<PAD>': 0, '<UNK>': 1}
for idx, (word, _) in enumerate(most_common, start=2):
    word2idx[word] = idx

idx2word = {idx: word for word, idx in word2idx.items()}

print(f"Vocabulary size: {len(word2idx)}")
print(f"Total tokens: {len(all_tokens)}")
print(f"Unique tokens: {len(token_counts)}")
print(f"\nMost common words:")
for word, count in most_common[:10]:
    print(f"  {word}: {count}")

## Step 5: Prepare Data

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

# Encode text to indices
def encode_text(text, word2idx, max_len=50):
    tokens = tokenize_konkani(text)
    indices = [word2idx.get(token, word2idx['<UNK>']) for token in tokens]
    
    # Pad or truncate
    if len(indices) < max_len:
        indices += [word2idx['<PAD>']] * (max_len - len(indices))
    else:
        indices = indices[:max_len]
    
    return indices

# Create label mapping
emotions = sorted(df['emotion'].unique())
label2idx = {label: idx for idx, label in enumerate(emotions)}
idx2label = {idx: label for label, idx in label2idx.items()}

print(f"Emotions: {emotions}")
print(f"Label mapping: {label2idx}")

# Encode all data
max_length = 50
X = [encode_text(text, word2idx, max_length) for text in df['text']]
y = [label2idx[emotion] for emotion in df['emotion']]

# Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")

# Dataset class
class EmotionDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.LongTensor(X)
        self.y = torch.LongTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataloaders
batch_size = 64
train_dataset = EmotionDataset(X_train, y_train)
val_dataset = EmotionDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

print(f"\n✅ Data prepared!")

## Step 6: Define Custom Model Architecture

In [ ]:
import torch.nn as nn

class KonkaniEmotionLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, num_layers=2, dropout=0.3):
        super(KonkaniEmotionLSTM, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # LSTM layers
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Attention layer
        self.attention = nn.Linear(hidden_dim * 2, 1)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Output layer
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
    
    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)  # (batch, seq_len, embedding_dim)
        
        # LSTM
        lstm_out, _ = self.lstm(embedded)  # (batch, seq_len, hidden_dim*2)
        
        # Attention
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        attended = torch.sum(attention_weights * lstm_out, dim=1)  # (batch, hidden_dim*2)
        
        # Dropout and output
        dropped = self.dropout(attended)
        output = self.fc(dropped)
        
        return output

# Model parameters
vocab_size = len(word2idx)
embedding_dim = 128
hidden_dim = 256
num_classes = len(emotions)
num_layers = 2
dropout = 0.3

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = KonkaniEmotionLSTM(vocab_size, embedding_dim, hidden_dim, num_classes, num_layers, dropout)
model = model.to(device)

print("="*70)
print("CUSTOM KONKANI EMOTION MODEL")
print("="*70)
print(f"Architecture: Bidirectional LSTM with Attention")
print(f"Vocabulary size: {vocab_size}")
print(f"Embedding dim: {embedding_dim}")
print(f"Hidden dim: {hidden_dim}")
print(f"Num classes: {num_classes}")
print(f"Num layers: {num_layers}")
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print("="*70)

## Step 7: Train Custom Model

In [ ]:
from tqdm import tqdm
import torch.optim as optim

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

num_epochs = 20
best_val_loss = float('inf')

print("="*70)
print("🚀 TRAINING CUSTOM KONKANI EMOTION MODEL")
print("="*70)
print()

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    
    for batch_x, batch_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += batch_y.size(0)
        train_correct += (predicted == batch_y).sum().item()
    
    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += batch_y.size(0)
            val_correct += (predicted == batch_y).sum().item()
    
    # Calculate metrics
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    train_acc = 100 * train_correct / train_total
    val_acc = 100 * val_correct / val_total
    
    print(f"\nEpoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    
    # Learning rate scheduling
    scheduler.step(val_loss)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
        }, '/kaggle/working/best_emotion_model.pt')
        print(f"  ✅ Best model saved!")

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)

## Step 8: Evaluate Model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load best model
checkpoint = torch.load('/kaggle/working/best_emotion_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Get predictions
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y in val_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.numpy())

# Classification report
print("="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(classification_report(all_labels, all_preds, target_names=emotions))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('/kaggle/working/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Confusion matrix saved!")

## Step 9: Save Complete Model

In [ ]:
import json
import os

# Create output directory
output_dir = '/kaggle/working/konkani_emotion_custom_model'
os.makedirs(output_dir, exist_ok=True)

# Save model
torch.save(model.state_dict(), f'{output_dir}/model.pt')

# Save vocabulary
with open(f'{output_dir}/vocab.json', 'w', encoding='utf-8') as f:
    json.dump(word2idx, f, ensure_ascii=False, indent=2)

# Save label mapping
with open(f'{output_dir}/labels.json', 'w') as f:
    json.dump({'label2idx': label2idx, 'idx2label': idx2label}, f, indent=2)

# Save config
config = {
    'vocab_size': vocab_size,
    'embedding_dim': embedding_dim,
    'hidden_dim': hidden_dim,
    'num_classes': num_classes,
    'num_layers': num_layers,
    'dropout': dropout,
    'max_length': max_length,
}
with open(f'{output_dir}/config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Model saved to:", output_dir)
print("\nFiles:")
!ls -lh {output_dir}

## Step 10: Test Model

In [ ]:
def predict_emotion(text, model, word2idx, idx2label, max_length=50):
    model.eval()
    
    # Encode text
    indices = encode_text(text, word2idx, max_length)
    x = torch.LongTensor([indices]).to(device)
    
    # Predict
    with torch.no_grad():
        output = model(x)
        probs = torch.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_idx].item()
    
    return idx2label[pred_idx], confidence

# Test samples
test_texts = [
    "हांव खूश आसा",
    "हांव दुखी आसा",
    "हांव रागीत आसा",
    "हांव भयभीत आसा",
]

print("="*70)
print("TESTING CUSTOM EMOTION MODEL")
print("="*70)

for text in test_texts:
    emotion, confidence = predict_emotion(text, model, word2idx, idx2label, max_length)
    print(f"\nText: {text}")
    print(f"Emotion: {emotion}")
    print(f"Confidence: {confidence:.4f}")

print("\n" + "="*70)
print("✅ CUSTOM KONKANI EMOTION MODEL READY!")
print("="*70)